In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook

In [2]:
def load_call_taxi(date):
    URL = f'http://openapi.seoul.go.kr:8088/676a7359436c696d33394451454277/xml/disabledCalltaxi/1/100/{date}'
    res = requests.get(URL)
    html = res.text
    soup = BeautifulSoup(html, 'lxml')
    
    no = []
    cartype = []
    receipttime = []
    settime = []
    ridetime = []
    startpos1 = []
    startpos2 = []
    endpos1 = []
    endpos2 = []
    
    for items in soup.findAll('item'):
        no.append(items.find('no').text)
        cartype.append(items.find('cartype').text.replace(' ', ''))
        receipttime.append(items.find('receipttime').text)
        settime.append(items.find('settime').text)
        ridetime.append(items.find('ridetime').text)
        startpos1.append(items.find('startpos1').text)
        startpos2.append(items.find('startpos2').text)
        endpos1.append(items.find('endpos1').text)
        endpos2.append(items.find('endpos2').text)
        
    dfs = pd.DataFrame({'차량고유번호':no,
                   '차량타입':cartype,
                   '예정일시':receipttime,
                   '배차일시':settime,
                   '승차일시':ridetime,
                   '출발지(구)':startpos1,
                   '출발지(동)':startpos2,
                   '목적지(구)':endpos1,
                   '목적지(동)':endpos2})
    
    return dfs
        

In [3]:
def GetDate(start, end):
    dateset = pd.date_range(start=start, end=end, freq='D')
    dates = []
    for date in dateset:
        dates.append(str(date)[:10])
    return dates


In [4]:
df = pd.DataFrame()
for date in tqdm_notebook(GetDate('2018-07-01', '2018-09-30')): # 날짜 수정(각 월에 맞게) !!! 
    df = pd.concat([df, load_call_taxi(date)], axis=0)




In [5]:
df.to_csv('call_taxi_raw_07.csv', encoding='euc-kr', index=False) # 파일 이름 수정( 01~12 )!!! 파일 경로 수정!!! 